In [1]:
import os
import logging
import pandas as pd
import getpass
from datetime import datetime
logger = logging.getLogger('LabJournal')
logging.basicConfig(level=logging.DEBUG)

In [2]:
import sys
sys.path.append("../../../") # need this if labjournal is not installed

from simdb.utils.fileFinder import find_files
from simdb.databaseModel import *
from simdb.databaseAPI import *
import simdb.utils.tpr_parser as tpr_parser


In [ ]:
SIMS = '/home/andrejb/Research/SIMS'
GMXBIN="/home/soft/gromacs/gromacs-2018/inst/shared/bin/"
os.environ['GMXBIN'] = GMXBIN

user = getpass.getuser()

In [ ]:
logger.info('create_database:create_database: FIND FILES')
ERRORS=False
WARNINGS=False

In [ ]:
%%time
SIM_IDS=[]
PATHS=[]
METAS=[]

for fname in find_files(pattern = 'topol.tpr', path = SIMS, dir_ignore = ['data']):
    path =  os.path.dirname(fname)
    sim_id = os.path.basename(path)
    try:
        meta = pd.Series.from_csv(os.path.join(path, "meta.csv"))
    except IOError:
        meta = pd.Series()
    SIM_IDS.append(sim_id)
    PATHS.append(path)
    METAS.append(meta)

In [ ]:
db = 'andrej_raw.db'
# remove old DB
try:
    os.remove(db)
    logger.info('create_database:create_database: removed old file: %s', db)
except:
    pass
engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()
setup_database(engine)

In [ ]:
%%time
for i, entry_id in enumerate(SIM_IDS):
    path = PATHS[i]
    tprfile = os.path.join(path,'topol.tpr')
    
    mapped_keywords = tpr_parser.main(tprfile)
    
    meta = METAS[i]
    try:
        description = meta['note']
        del meta['note']
    except KeyError:
        description = ""
    
    if 'created_on' in meta:
        created_on  = meta['created_on']
        del meta['created_on']
    else:
        created_on = datetime.fromtimestamp(os.path.getmtime(tprfile))
        
    sim = store_dict(
        entry_id    = entry_id,
        path        = path,
        sim_type    = "GROMACS",
        description = description,
        created_on  = created_on,
        owner       = user,
        raw_mdp_parameters = mapped_keywords,
        raw_keywords = meta,
    )
    
    session.add(sim)
    session.flush()

session.commit()
session.close()

In [ ]:
engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
session.query(Main).all()

In [ ]:
session.close()

## Add groups to DB

In [ ]:
db = 'test.db'

engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()
setup_database(engine)

In [ ]:
sim1 = Main(entry_id = "sim01")
sim2 = Main(entry_id = "sim02")

sim1.parents.append(sim2)

In [ ]:
#construct a category and add some products to it
oGroup = Groups(name="Test Group")

sim1 = Main(entry_id = "sim01")
sim1.groups.append(oGroup)
# oGroup.entries.append(sim1)

# # interact with products from an existing category
# for oProduct in oCat.products:
#     print oProduct.name
    
# #interact with categories of an existing product
# oProduct = DBSession.query(Product).filter_by(name="")
# for oCat in oProduct.categories:
#     print oCat.name

In [ ]:
entry[-1]

In [ ]:
session.delete(entry[-1])

In [ ]:
session.commit()

In [ ]:
session.close()

In [ ]:
session.query(Main).first()

In [3]:
db_path = "andrej_raw.db"
table = getEntryTable(db_path)

In [4]:
sel = table[selectByKeyword(table, "linker", "k48")]

In [5]:
s = openDatabase(db_path)

In [6]:
entry = s.query(Main)[-1]

In [10]:
s.delete(entry)
s.commit()

OperationalError: (OperationalError) no such table: association_main_groups u'SELECT groups.id AS groups_id, groups.name AS groups_name \nFROM groups, association_main_groups \nWHERE ? = association_main_groups.main_id AND groups.id = association_main_groups.group_id' (699,)

In [ ]:
s.query(Main).filter(Main.entry_id == sel.entry_id).first()

In [ ]:
def addGroup(db_path, table, groupname):
    """Add all simulations in table to group,"""
    
    # open databae
    s = openDatabase(db_path)
    
    for i, sim in table.iterrows():
        
        # get simulation
        sim = s.query(Main).filter(Main.entry_id == sim.entry_id).one()
        
        # create group and add to DB
        group = Groups(main_id = sim.id, name=groupname)
        s.add(group)
    
    # commit and close
    s.commit()
    s.close()

In [ ]:
def getEntryGroup(db_path, groupname):
    

In [ ]:
addGroup(db_path, table, "K48 linked")

## Setup an empty DB

In [ ]:
db = 'empty.db'
# remove old DB
try:
    os.remove(db)
    logger.info('create_database:create_database: removed old file: %s', db)
except:
    pass
engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()
setup_database(engine)
session.close()